# TG3: использование моделей

# Пример бота: разметка

Предположим у вас есть тысячи отзывов на кинофильмы, и вам нужно их разметить как "положительные", "отрицательные" или "нейтральные". Вы можете сделать бота, который будет отправлять юзеру какой-то отзыв из нашего списка и клавиатуру с кнопками "+", "-", "=", а затем бот будет записывать ответы. Бота мы можем отправить друзьям, чтобы они помогли нам с разметкой. =)

Результаты мы будем записывать в файл `results.csv`, а хранить служебную информацию о том, кто разметчик и какие отзывы мы ему прислали, будем складывать "на полку" — [shelve](https://docs.python.org/3/library/shelve.html). Это встроенная питоновская библиотека, позволяющая создавать мини-базу данных, где можно хранить файлы в виде "ключ-значение". Ничто не мешает нам одновременно с этим записывать id разметчиков в файл с результатами.

[Файл `reviews.csv`](https://github.com/hse-ling-python/seminars/blob/master/chatbots/telegram/reviews.csv)

In [ ]:
# -*- coding: utf-8 -*-
# импортируем модули
import telebot
import conf
import random
import shelve
from telebot import types


bot = telebot.TeleBot(conf.TOKEN)

# предположим, отзывы у нас хранятся в виде csv-файла с номерами отзывов и собственно текстами
with open('reviews.csv', 'r', encoding='utf-8') as f:
    reviews = {}  # создадим словать отзывов
    for line in f:
        num, text = line.strip().split('\t')
        reviews[num] = text
review_keys = list(reviews.keys())  # и отдельно массив ключей


# собираем клавиатуру для разметки (возможно имеет смысл добавить кнопку "не знаю"?)
keyboard = types.ReplyKeyboardMarkup(row_width=3)
btn1 = types.KeyboardButton('+')
btn2 = types.KeyboardButton('-')
btn3 = types.KeyboardButton('=')
keyboard.add(btn1, btn2, btn3)


# shelve используется как мини-база данных, там можно хранить файлы в виде "ключ-значение"
# в этой базе мы будем хранить информацию о том, какой отзыв мы недавно прислали юзеру
shelve_name = 'shelve.db'  # Файл с хранилищем

def set_user_review(chat_id, review):
    """
    Записываем юзера в базу данных и запоминаем номер отзыва, который мы ему дали
    """
    with shelve.open(shelve_name) as storage:
        storage[str(chat_id)] = review


def get_user_review(chat_id):
    """
    Вспоминаем, какой отзыв мы отправили на разметку
    :return: (str) Номер отзыва / None
    """
    with shelve.open(shelve_name) as storage:
        try:
            review = storage[str(chat_id)]
            return review
        # Если человека нет в базе, ничего не возвращаем
        except KeyError:
            return None

# этот обработчик запускает функцию send_welcome, когда пользователь отправляет команду /help
@bot.message_handler(commands=['help'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Здравствуйте! Это бот для разметки отзывов на кинофильмы.\n Положительные отзывы отмечаются плюсом +, отрицательные минусом -, а нейтральные знаком равно =.")


# этот обработчик запускает функцию, когда пользователь отправляет команду /start
@bot.message_handler(commands=['start'])
def send_first_review(message):
    review_num = random.choice(review_keys)
    bot.send_message(message.chat.id, reviews[review_num], reply_markup=keyboard)
    set_user_review(message.chat.id, review_num)


@bot.message_handler(regexp='[-+=]')  # этот обработчик реагирует на символы разметки и записывает результаты
def get_answer(message):
    review_num = get_user_review(message.chat.id)  # проверяем, есть ли юзер в базе данных
    if review_num:
        # если есть, открываем файл с результатами и записываем туда разметку
        with open('results.csv', 'a', encoding='utf-8') as results:
            results.write(review_num + '\t' + message.text + '\n')
        # и сразу отправляем новый отзыв
        review_num = random.choice(review_keys)
        bot.send_message(message.chat.id, reviews[review_num], reply_markup=keyboard)
        set_user_review(message.chat.id, review_num)
    else:
        # если нет, то что-нибудь говорим об этом
        bot.send_message(message.chat.id, 'Вы не разметили отзыв.')

In [ ]:
if __name__ == '__main__':
    bot.polling(none_stop=True)


# Марковские цепи

> **Цепь Маркова** *(англ. Markov chain)* — последовательность случайных событий с конечным или счётным числом исходов, характеризующаяся тем, что при фиксированном настоящем будущее независимо от прошлого. Процесс в каждый момент времени находится в одном из состояний.

[Вот хороший тьюториал](https://tproger.ru/translations/markov-chains/), где подробно (с картинками!) описан процесс работы этого алгоритма и дан пример написания генератора текста на основе цепи Маркова с нуля. В этом полезно разобраться на досуге, хотя все уже сделано за нас. Есть несколько питоновских библиотек для генерации текста с помощью марковских цепей — например, `markovify`. [Вот ее документация](https://github.com/jsvine/markovify).

In [ ]:
!pip install markovify

In [ ]:
import pandas as pd

provs = pd.read_csv('dal_proverbs.csv', sep='\t', encoding='utf-8')
provs.head()

,proverb,topic
0,Жить — богу служить.,Бог — Вера
1,"Кто велик, яко бог наш (Влад. Моном.).",Бог — Вера
2,"Не нам, не нам, но имени твоему (т. е. слава).",Бог — Вера
3,Велико имя господне на земли.,Бог — Вера
4,"В мале бог, и в велике бог.",Бог — Вера


In [ ]:
shuffled = provs.sample(frac=1)
train = ' '.join(shuffled.proverb)
train[:200]

'Будет он меня помнить. В поле съезжаются, родом не считаются (от местничества). На безлюдье и Фома дворянин. На безлюдье и сидни в честь. В Москве все найдешь, кроме птичьего молока. Гусаром игры не з'

In [ ]:
import markovify

m = markovify.Text(train)

In [ ]:
for i in range(5):
    print(m.make_sentence())

Не думай, как бы взять, а никто не пришлет, а с должником расплатись!
Хоть тресни синица, а не видена золотая.
Не учили, покуда поперек лавки ложился; а во сне грезится.
Не загадывай в год, а завтра — свиней пасти.
Что поставят, то глядим.


In [ ]:
for i in range(5):
    print(m.make_short_sentence(max_chars=100))

Даст бог счастье — на кокорник, в среду встал да палку взял, тот и ветху рад.
Жить в соседах — быть без грибов.
Запрос в карман посадил.
Полюбил его, как собака с волком не травится, поп попом не судится.
Покорился да в разуме тверд.


## Задание

Напишите простого бота, который присылет пользователю сгенерированную с помощью марковской цепи фразу. Пользователь может ввести желаемую длину фразы, а если он этого не сделал, то шлем фразу произвольной длины. Обучить марковскую цепь можно на чем угодно. Чем больше будет объем обучающих данных, тем интереснее будут результаты! А чтобы было не скучно, можно смешать несколько разных по тематике и стилю текстов.

[Пример реализации](https://github.com/hse-ling-python/seminars/tree/master/different_api/basic_bot)

# Пример бота с несколькими моделями

https://github.com/hse-ling-python/seminars/tree/master/chatbots/ml_bot